In [5]:
import redis
import json
from functools import partial
import numpy as np
# IMPORTANT: Run redis-server in a shell window for it to work.
stateDB = redis.Redis(host='localhost', port=6379, db=0)
# Just making a default entry into database for state for epsilon greedy algorithm
rewards = np.zeros((3,))
stateDB.hset("state1", "rewards", str(rewards)[1:-1])
stateDB.hset("state1", 'epsilon', 0.5)
def selection_policy(state):
    epsilon = load_var(state, 'epsilon', float)
    print('Epsilon:', epsilon)
    rewards = load_var(state, 'rewards', partial(np.fromstring, sep=' '))
#     return 0
    if np.random.rand() < epsilon:
        return np.random.randint(0, 3)
    else:
        return np.argmax(rewards)
def gen_res(model_id, inp):
    return {model_id: model_id * inp + 1}
def combine(state, stateDB, results, dbName):
    epsilon = load_var(state, 'epsilon', float)
    rewards = load_var(state, 'rewards', partial(np.fromstring, sep=' '))
    for model, result in results.items():
        rewards[model] = result
    print('Rewards:', rewards)
    epsilon = 0.9*epsilon
    stateDB.hset(dbName, 'epsilon', epsilon)
    stateDB.hset(dbName, 'rewards', str(rewards)[1:-1])
    return max(results.values())
# Helper function to deal with redis converting everything to byte strings
def load_var(state, var_name, type_cast_func):
    return type_cast_func(state[var_name.encode('utf-8')])
counts = np.zeros((3,))
def qf(query):
    q = json.loads(query)['input'] # Will be a list of stuff
    state = stateDB.hgetall('state1') # Will be a dic of state variables
    model_id = selection_policy(state)
    print("Models Selected", model_id)
    counts[model_id] += 1
    results = gen_res(model_id, q)
    return combine(state, stateDB, results, 'state1')

ConnectionError: Error 61 connecting to localhost:6379. Connection refused.

In [6]:
for i in range(30):
    q = json.dumps({'input': i})
    print(qf(q))
print(counts / 30)

NameError: name 'qf' is not defined